<a href="https://colab.research.google.com/github/jennahgosciak/nyc_fire_risk/blob/main/00_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
import config
import quickstart

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "1_raw")
out= os.join(root, "data", "2_intermediate")

## Load data from google drive

In [ ]:
# left join from pluto to capture all properties
# use df for space constraints
vac_pluto_df= pluto_df.merge(vac, how = "left", on = "bbl", indicator = True)
print(vac_pluto_df["_merge"].value_counts())

# create indicator of fire vacancy based on indicator
vac_pluto_df["vacate_ind"]= np.where(
            vac_pluto_df["_merge"]=="both", 1, 0)

print("\nTab of outcome var (vacate ind)")
print(vac_pluto_df[["vacate_ind", "_merge"]].value_counts())
vac_pluto_df.drop("_merge", axis = 1, inplace = True)

In [ ]:
print(vac_ew.shape)
# check size of unmerged buildings (small buildings under 10,000 sq. ft. for gov't buildings
# and 25,000 shouldn't be merging)
print(vac_ew.loc[vac_ew["property_id"].notna(), :].shape)
print(vac_ew.loc[vac_ew["property_id"].isna(), :].shape)
print(vac_ew.loc[(vac_ew["property_id"].isna()) & \
                       (vac_ew["bldgarea"] < 25000), "bldgarea"].shape)

print("\nMerge rate:")
print(vac_ew["_merge"].value_counts())
# assert there are no values unmerged on rhs
assert vac_ew.loc[vac_ew["_merge"]=="right_only"].shape[0]==0
vac_ew.drop("_merge", axis = 1, inplace = True)